In [1]:
%pip install -U gymnasium
%pip install -U gymnasium[atari,accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.5 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=a9ac9d05696f90fcb4bfd1dca17e09b6db50ecdc4bae390c8e5ae1458a7be2f3
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
# import gymnasium as gym

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import gym
import torch.nn.functional as F
import argparse

In [3]:
env = gym.make('ALE/Pong-v5')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
from gym.wrappers import AtariPreprocessing

In [ ]:
# env = gym.make('ALE/Pong-v5')
# env = AtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, noop_max=30)

/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
# # Preprocessing Frames and Stacking Observations
# class FrameStackingWrapper(gym.Wrapper):
#     def __init__(self, env, obs_stack_size):
#         super(FrameStackingWrapper, self).__init__(env)
#         self.obs_stack_size = obs_stack_size
#         self.obs_stack = None

#     def reset(self, **kwargs):
#         obs = self.env.reset(**kwargs)
#         self.obs_stack = torch.cat([obs.unsqueeze(0)] * self.obs_stack_size, dim=0)
#         return self.obs_stack

#     def step(self, action):
#         next_obs, reward, done, info = self.env.step(action)
#         self.obs_stack = torch.cat((self.obs_stack[1:, ...], next_obs.unsqueeze(0)), dim=0)
#         return self.obs_stack, reward, done, info

# env = FrameStackingWrapper(env, obs_stack_size=4)  # Assuming 4 frames for frame stacking
# state_size = env.observation_space.shape[0]
# print("state size", state_size)

In [ ]:
# # Install necessary dependencies
# !apt-get install -y xvfb x11-utils
# !pip install gym[atari]
# !pip install pyvirtualdisplay

In [ ]:
# # Create a virtual display
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()

In [ ]:

class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def __len__(self):
        return len(self.memory)

    def push(self, obs, action, next_obs, reward, donne):
        if len(self.memory) < self.capacity:
            self.memory.append(None)

        self.memory[self.position] = (obs, action, reward, next_obs, donne)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        """
        Samples batch_size transitions from the replay memory and returns a tuple
            (obs, action, next_obs, reward)
        """
        sample = random.sample(self.memory, batch_size)
        return zip(*sample)

# Initialize environment and agent
# env = gym.make('CartPole-v1')
env = gym.make('ALE/Pong-v5')
env = AtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, noop_max=30)

# state_size = env.observation_space.shape
# print("state size", state_size)
# env = gym.wrappers.RecordVideo(env, './video/', episode_trigger=lambda episode_id: True)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n




CartPole = {
    'Observation_stack_size': 4,
    'memory_size': 10000,
    'n_episodes': 100,
    'batch_size': 32,
    'target_update_frequency': 100,
    'train_frequency': 1,
    'gamma': 0.95,
    'lr': 1e-4,
    'eps_start': 1.0,
    'eps_end': 0.05,
    'anneal_length': 10**6,
    'n_actions': 2,
    'epsilon': 0.9
}
ENV_CONFIGS = {
    'CartPole-v1': CartPole
}

env_config = ENV_CONFIGS['CartPole-v1']
replay_memory = ReplayMemory(env_config['memory_size'])


# Define the Q-network
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        # self.fc1 = nn.Linear(state_size, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, action_size)
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, action_size)

        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
    def forward(self, x):
        # x = torch.cat([x] * 4, dim=0)
        x = x.squeeze(1)

        # x = x[:4, :, :]

        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        # print('x: ',x.shape)
        x = self.flatten(x)
        # print('x flatten: ',x.shape)
        # x = x.reshape(1, 3136)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        # print('x reshape: ',x.shape)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
q_network = QNetwork(state_size, action_size).to(device)
target_network = QNetwork(state_size, action_size).to(device)
target_network.load_state_dict(q_network.state_dict())
target_network.eval()



# Define the DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size, lr=0.0001, gamma=0.95, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        #self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        #self.q_network = QNetwork(state_size, action_size).to(self.device)
        #self.target_network = QNetwork(state_size, action_size).to(self.device)
        #self.target_network.load_state_dict(self.q_network.state_dict())
        #self.target_network.eval()

        self.optimizer = optim.Adam(q_network.parameters(), lr=lr)
        self.loss_function = F.mse_loss

    def act(self, state, exploit = False):
       if exploit == False:
          if random.random() > self.epsilon:
              with torch.no_grad():
                  # state = torch.FloatTensor(state).unsqueeze(0).to(device)
                  state = torch.tensor(state, dtype=torch.float32, device=device)
                  q_values = q_network(state)
                  action = q_values.argmax().item()
          else:
              action = random.randrange(self.action_size)
          return action
       else:
         with torch.no_grad():
                  # state = torch.FloatTensor(state).unsqueeze(0).to(device)
                  state = torch.tensor(state, dtype=torch.float32, device=device)
                  q_values = q_network(state)
                  action = q_values.argmax().item()
                  return action


    def optimize(self, state, action, reward, next_state, done, batch_size=32, step = 1):
        if len(replay_memory) < batch_size:
            return

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # batch = random.sample(replay_memory, batch_size)
        # states, actions, rewards, next_states, dones = zip(*batch)
        states, actions, next_states, rewards, dones = replay_memory.sample(batch_size)
        # print('states: ',states)
        states = torch.stack(states).squeeze(1).to(device)
        # print('states stack size: ',states.shape)
        next_states = torch.stack(next_states).squeeze(1).squeeze(1)
        # print("states",states.shape)
        # states = torch.FloatTensor(states).to(device)
        states = torch.tensor(states, dtype=torch.float32, device=device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(device)
        # rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
        # next_states = torch.FloatTensor(next_states).to(device)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
        # dones = torch.FloatTensor(dones).unsqueeze(1).to(device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device)
        # print("states:", states.shape)
        # print("actions:", actions.shape)
        # print("rewards:", rewards.shape)
        # print("next_states:", next_states.shape)
        # print("dones:", dones.shape)

        # print("q network: ", q_network(states))
        q_values = q_network(states).gather(1, actions)
        next_q_values = target_network(next_states).max(dim=1, keepdim=True)[0].detach()
        target_q_values = rewards + self.gamma * next_q_values * (1 - dones)

        loss = self.loss_function(q_values, target_q_values)
        # print("loss: ", loss)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        if step % env_config['target_update_frequency']:
            target_network.load_state_dict(q_network.state_dict())

        # Update epsilon
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)



agent = DQNAgent(state_size, action_size)
# Training parameters
# n_episodes = 25
batch_size = 32

args = argparse.Namespace(
    env='ALE/Pong-v5',
    evaluate_freq=25,
    evaluation_episodes=1000
)

def evaluate_policy(agent, env, env_config, args, n_episodes, render=False, verbose=False):
    """Runs {n_episodes} episodes to evaluate current policy."""
    total_return = 0
    for i in range(n_episodes):
        state = env.reset()
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        obs_stack = torch.cat(env_config['Observation_stack_size'] * [state]).unsqueeze(0).to(device)
        #state = preprocess(obs, env=args.env).unsqueeze(0)
        # print("state:", state)
        # print("state shape:", state.shape)

        terminated = False
        episode_return = 0
        truncated = False
        while not terminated and not truncated:
            if render:
                env.render()

            action = agent.act(obs_stack, exploit=True)
            next_state, reward, terminated, truncated = env.step(action)
            #obs = preprocess(obs, env=args.env).unsqueeze(0)
            obs_stack = obs_stack.to(device)
            next_state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0).to(device)
            next_state = torch.cat((obs_stack[:, 1:, ...], next_state.unsqueeze(1)), dim=1).to(device)
            # print('next_state')
            # state = obs_stack


            episode_return += reward

        total_return += episode_return

        if verbose:
            print(f'Finished episode {i+1} with a total return of {episode_return}')


    return total_return / n_episodes

# print('ob stac size',env_config['Observation_stack_size'])
for episode in range(env_config['n_episodes']):
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    # print('obs stack', state.shape)
    total_reward = 0
    step = 1
    # obs_stack_size = torch.stack(state).shape
    obs_stack = torch.cat(env_config['Observation_stack_size'] * [state]).unsqueeze(0).to(device)
    # print('obs stack lkk', obs_stack[:, 1:, ...])
    while True:
        action = agent.act(obs_stack)
        next_state, reward, done, _ = env.step(action)
        # print(done, truncated)

        next_state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        next_state = next_state.to(device)
        obs_stack = obs_stack.to(device)
        next_state = torch.cat((obs_stack[:, 1:, ...], next_state.unsqueeze(1)), dim=1).to(device)
        # print('next_state')
        state = obs_stack
        # print('obs_state', state.shape)

        replay_memory.push(state, action, reward, next_state, done)
        # print("reward: ", reward)
        # print("states:", state.shape)#[1,4,84,84]
        # print("actions:", action)
        # print("rewards:", reward)
        # print("next_states:", next_state.shape) #[1,4,84,84]
        # print("dones:", done)


        if step%env_config["train_frequency"] == 0:
          # print('step',step)
          agent.optimize(state, action, reward, next_state, done, batch_size, step)
        total_reward += reward
        step += 1
        # print('step',step)
        state = next_state
         # Render the environment
        # env.render()
        if done:
            break
    # env.close()

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")
    if episode % args.evaluation_episodes == 0:
      print(evaluate_policy(agent, env, env_config, args, args.evaluation_episodes))

env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a 

Episode: 1, Total Reward: -21.0


<ipython-input-14-5f0e359163c7>:137: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32, device=device)


0.0
Episode: 2, Total Reward: -21.0
Episode: 3, Total Reward: -21.0
Episode: 4, Total Reward: -21.0
Episode: 5, Total Reward: -21.0
Episode: 6, Total Reward: -21.0
Episode: 7, Total Reward: -20.0
Episode: 8, Total Reward: -19.0
Episode: 9, Total Reward: -21.0
Episode: 10, Total Reward: -21.0
Episode: 11, Total Reward: -21.0
Episode: 12, Total Reward: -20.0
Episode: 13, Total Reward: -21.0
Episode: 14, Total Reward: -21.0
